In [6]:
import os
import pandas as pd
from utils import partial_match_scores


root = "../datasets/myriadlama/llama3.2_1b_it"
root = "../datasets/myriadlama/llama3.2_3b_it"
root = "../datasets/myriadlama/qwen2.5_3b_it"
# root = "../datasets/myriadlama/qwen2.5_7b_it"
# root = "../datasets/myriadlama/llama3.1_8b_it"

confi_df = pd.read_feather(os.path.join(root, "confidence.feather"))
confi_df["sample_lemmas"] = confi_df["sample_lemmas"].apply(lambda xs: [list(x) for x in xs])
confi_df["answer_lemmas"] = confi_df["answer_lemmas"].apply(lambda xs: [list(x) for x in xs])


In [7]:
from gurobipy import and_
from sklearn import ensemble
from utils import is_matched_str, partial_match

droot = os.path.join(root, "diversity")

for i in range(2, 5):
    ensemble_scores = []
    consistency_scores = []
    or_matches_scores = []
    diversity_scores = []

    print(f"Processing ensemble {i}...")
    for fn in os.listdir(droot):
        if not fn.endswith(".feather"):
            continue

        if len(fn.split(",")) != i:
            continue
        df = pd.read_feather(os.path.join(droot, fn))
        df["answer_lemmas"] = df["answer_lemmas"].apply(lambda xs: [list(x) for x in xs])
        
        # Compute partial match scores
        scores = partial_match_scores(df['predict_lemma'].tolist(), df["answer_lemmas"].tolist())
        ensemble_scores.append(scores)

    print("Number of ensemble scores:", len(ensemble_scores), f"Average ensemble score: {sum(ensemble_scores) / len(ensemble_scores):.3f}")

Processing ensemble 2...
Number of ensemble scores: 10 Average ensemble score: 0.420
Processing ensemble 3...
Number of ensemble scores: 10 Average ensemble score: 0.433
Processing ensemble 4...
Number of ensemble scores: 5 Average ensemble score: 0.443
